<h1>Wyszukiwanie informacji</h1>

Jednym z zadań nlp jest wyszukiwanie dokumentów podobnych dla danego zapytania (wyszukiwarki internetowe).

Weźmy korpus dokumentów, jak poniżej. 

In [1]:
corpus = ["Romeo and Juliet",
         "Juliet: O happy dagger",
         "Romeo died by dagger",
         "'Live free or die', that’s the New-Hampshire’s motto",
         "Did you know, New-Hampshire is in New-England"]

ZADANIE: Zadajemy zapytanie "dies, dagger". Prosze uszeregować dokumenty w kolejności od najbardziej pasujących. 

Transformujemy query do naszej reprezentacji i liczymy ile słów się pokrywa (mnożenie wektorów i sumowanie)



In [2]:
query = ["dies", "dagger"]

In [4]:
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer


def my_tokenizer(sent):
    tokens = word_tokenize(sent)
    stemmer = PorterStemmer()
    res = [stemmer.stem(word) for word in tokens]
    return res 

vectorizer = CountVectorizer(tokenizer = my_tokenizer, stop_words=stopwords.words('english') + list(string.punctuation))
vectorizer.fit(corpus)
print(vectorizer.vocabulary_)
data = vectorizer.transform(corpus).todense()
print(data)

query_data = vectorizer.transform(query).todense()
print(query_data)



{'romeo': 10, 'juliet': 5, 'happi': 4, 'dagger': 1, 'die': 2, "'live": 0, 'free': 3, '’': 11, 'new-hampshir': 9, 'motto': 7, 'know': 6, 'new-england': 8}
[[0 0 0 0 0 1 0 0 0 0 1 0]
 [0 1 0 0 1 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 1 0]
 [1 0 1 1 0 0 0 1 0 1 0 2]
 [0 0 0 0 0 0 1 0 1 1 0 0]]
[[0 0 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]]


In [78]:
import numpy as np
res = np.sum(np.dot(data, query_data.T), axis=1)
res

matrix([[0],
        [1],
        [2],
        [1],
        [0]], dtype=int64)

Żeby dokumenty były podobne to muszą zawierać te same słowa.

ZADANIE: Czy aby na pewno 2 i 4 są tak samo podobne do query?



Można użyć transformaty tfidf: https://en.wikipedia.org/wiki/Tf%E2%80%93idf.

tf - waga słowa jest proporcjonalna do ilości jego wystąpień w dokumencie

idf - waga jest odwrotnie proporcjonalna do ilości dokumentów w jakich występuje

Intuicyjnie tfidf zmniejsza wagi często występujących słów, co powoduje że są mniej ważne (potencjalnie można nie usuwać stopwords..)

ZADANIE: Proszę spróbować zaimplementować własne tfidf (https://medium.freecodecamp.org/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3).


In [ ]:
#TODO

ZADANIE: Użyjmy Tfidf z sklearn i dokonajmy wyszukiwania.

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, stop_words=stopwords.words('english') + list(string.punctuation))
vectorizer.fit(corpus)
print(vectorizer.vocabulary_)
data = vectorizer.transform(corpus)
print(data.todense())



{'romeo': 10, 'juliet': 5, 'happi': 4, 'dagger': 1, 'die': 2, "'live": 0, 'free': 3, '’': 11, 'new-hampshir': 9, 'motto': 7, 'know': 6, 'new-england': 8}
[[ 0.          0.          0.          0.          0.          0.70710678
   0.          0.          0.          0.          0.70710678  0.        ]
 [ 0.          0.53177225  0.          0.          0.659118    0.53177225
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.57735027  0.57735027  0.          0.          0.          0.
   0.          0.          0.          0.57735027  0.        ]
 [ 0.34706676  0.          0.28001128  0.34706676  0.          0.          0.
   0.34706676  0.          0.28001128  0.          0.69413353]
 [ 0.          0.          0.          0.          0.          0.
   0.61418897  0.          0.61418897  0.49552379  0.          0.        ]]


In [81]:
query_data = vectorizer.transform(query)
print(query_data.todense())

res = np.sum(np.dot(data, query_data.T).todense(),axis=1)
res

[[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


matrix([[ 0.        ],
        [ 0.53177225],
        [ 1.15470054],
        [ 0.28001128],
        [ 0.        ]])

ZADANIE: Co spowodowało że podobieństwo zostało rozróżnione?